In [2]:
import sys
sys.path.append("../pycwl/")
import pycwl

In [3]:
multiply_clt = pycwl.CommandLineTool(baseCommand = "multiply.py")
multiply_clt.inputs["mlt_nr1"]= pycwl.IO(datatype = "int", default_value=5)
multiply_clt.inputs["mlt_nr2"]= pycwl.IO(datatype = "int")
multiply_clt.outputs["multiply_result"] = pycwl.IO(datatype = "int")
multiply_clt.arguments = [multiply_clt.inputs["mlt_nr1"],multiply_clt.inputs["mlt_nr2"]]

In [4]:
print multiply_clt.baseCommand
print multiply_clt.inputs
print multiply_clt.outputs
print multiply_clt.arguments

multiply.py
{'mlt_nr2': <pycwl.IO object at 0x0000000004ABA978>, 'mlt_nr1': <pycwl.IO object at 0x0000000004ABA940>}
{'multiply_result': <pycwl.IO object at 0x0000000004ABA8D0>}
[<pycwl.IO object at 0x0000000004ABA940>, <pycwl.IO object at 0x0000000004ABA978>]


In [5]:
add_clt = pycwl.CommandLineTool(baseCommand = "add.py")
add_clt.inputs["add_nr1"]= pycwl.IO(datatype = "int", default_value=5)
add_clt.inputs["add_nr2"]= pycwl.IO(datatype = "int")
add_clt.outputs["add_result"] = pycwl.IO(datatype = "int")
add_clt.arguments = [add_clt.inputs["add_nr1"],add_clt.inputs["add_nr2"]]

In [ ]:





add_clt = clt_node( baseCommand="add.py")
...



calc_flow = Workflow()#, inputs=[], outputs)
calc_flow.inputs["flow_nr1"] = IO(datatype=pyCWL.dtype.int)
calc_flow.inputs["flow_nr2"] = IO(datatype=pyCWL.dtype.int)
calc_flow.output["add/add_result"] = IO(datatype=pyCWL.dtype.int)

step = Step(multiply_clt)
step.input_map = {"flow_nr1": "mlt_nr_1",
              "flow_nr2": "mlt_nr2"}
step.output = {"multiply_result"}
calc_flow.steps["multiply"] = step

calc_flow.steps["add"] = Step(
    add_clt,
    input_map={
        "multiply/multiply_result": "add_nr_1",
        "flow_nr3": "add_nr2",
    },
    output={
        "add_result",
    },
)

